<a href="https://colab.research.google.com/github/gustavocmiranda/DS_Projects/blob/main/CBLOL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import matplotlib.offsetbox as offsetbox

from PIL import Image

from bs4 import BeautifulSoup
from urllib.request import urlopen

from urllib.request import Request
hdr = {'User-Agent': 'Mozilla/5.0'}

In [2]:
all_games_data = pd.DataFrame()
url = 'https://gol.gg/tournament/tournament-matchlist/CBLOL%20Split%201%202024/'

soup = BeautifulSoup(urlopen(url), 'lxml')

html_tables = pd.read_html(str(soup), header= 0)
df = html_tables[0]
df.rename(columns={'Unnamed: 1': 'Team 1', 'Unnamed: 3':'Team 2', 'Unnamed: 4': 'WEEK'}, inplace=True)

filtered_links = soup.find_all('a', href=lambda href: 'page-game' in href or 'page-preview' in href)
hs = []
for link in filtered_links:
  href = link.get('href').replace('..', '')
  hs.append(href)

df['href'] = pd.Series(hs)
df = df.dropna().reset_index().drop('index', axis=1)

for index, row in df.iterrows():
  href = row['href'].replace('-game', '-fullstats')
  team1 = row['Team 1']
  team2 = row['Team 2']
  week = row['WEEK'].replace('WEEK', '')
  url = f'https://gol.gg{href}'

  req = Request(url,headers=hdr)
  page = urlopen(req)
  table_soup = BeautifulSoup(page, 'lxml')
  html_tables = pd.read_html(str(table_soup), index_col=0, header= 0)
  temp = html_tables[0]
  temp = temp.transpose().reset_index()

  table = table_soup.find_all('table')[0]
  images = table.findAll('img')
  champions = []
  for image in images:
    champions.append(image.get('alt'))
  champions = pd.Series(champions)
  temp['index'] = champions
  temp['Team'] = np.where(temp.index < 5, team1, team2)
  temp['Side'] = np.where(temp.index < 5, 'Blue', 'Red')
  temp['Week'] = week
  temp.rename(columns={'index':'Champion'}, inplace=True)
  temp.drop('CS in Enemy Jungle', axis=1, inplace=True)


  all_games_data = pd.concat([all_games_data, temp], axis=0, ignore_index=True)


#all_games_data.reset_index(inplace=True)
#all_games_data.drop('index', inplace=True, axis=1)
all_games_data['Solo kills'].fillna('0', inplace=True)
all_games_data[['Level', 'Kills', 'Deaths', 'Assists','CS', "CS in Team's Jungle", 'CSM', 'Golds', 'GPM', 'Vision Score', 'Wards placed', 'Wards destroyed',
       'Control Wards Purchased', 'Detector Wards Placed', 'VSPM', 'WPM', 'VWPM', 'WCPM', 'Total damage to Champion', 'Physical Damage',
       'Magic Damage', 'True Damage', 'DPM', 'K+A Per Minute', 'Solo kills', 'Double kills', 'Triple kills', 'Quadra kills',
       'Penta kills', 'GD@15', 'CSD@15', 'XPD@15', 'LVLD@15',
       'Objectives Stolen', 'Damage dealt to turrets',
       'Damage dealt to buildings', 'Total heal', 'Total Heals On Teammates',
       'Damage self mitigated', 'Total Damage Shielded On Teammates',
       'Time ccing others', 'Total Time CC Dealt', 'Total damage taken',
       'Total Time Spent Dead', 'Consumables purchased', 'Items Purchased',
       'Shutdown bounty collected', 'Shutdown bounty lost', 'Week']] = all_games_data[['Level', 'Kills', 'Deaths', 'Assists','CS', "CS in Team's Jungle", 'CSM', 'Golds', 'GPM', 'Vision Score', 'Wards placed', 'Wards destroyed',
       'Control Wards Purchased', 'Detector Wards Placed', 'VSPM', 'WPM', 'VWPM', 'WCPM', 'Total damage to Champion', 'Physical Damage',
       'Magic Damage', 'True Damage', 'DPM', 'K+A Per Minute', 'Solo kills', 'Double kills', 'Triple kills', 'Quadra kills',
       'Penta kills', 'GD@15', 'CSD@15', 'XPD@15', 'LVLD@15',
       'Objectives Stolen', 'Damage dealt to turrets',
       'Damage dealt to buildings', 'Total heal', 'Total Heals On Teammates',
       'Damage self mitigated', 'Total Damage Shielded On Teammates',
       'Time ccing others', 'Total Time CC Dealt', 'Total damage taken',
       'Total Time Spent Dead', 'Consumables purchased', 'Items Purchased',
       'Shutdown bounty collected', 'Shutdown bounty lost', 'Week']].astype('float')
all_games_data['KDA'] = np.where(all_games_data['KDA'] == 'Perfect KDA', all_games_data['Kills']+all_games_data['Assists'], all_games_data['KDA'])
all_games_data['KDA'] = all_games_data['KDA'].astype('float')

In [3]:
all_games_data

,Champions,Player,Role,Level,Kills,Deaths,Assists,KDA,CS,CS in Team's Jungle,...,Total Time CC Dealt,Total damage taken,Total Time Spent Dead,Consumables purchased,Items Purchased,Shutdown bounty collected,Shutdown bounty lost,Team,Side,Week
0,K,Kiari,TOP,15.0,1.0,3.0,0.0,0.3,242.0,0.0,...,1309.0,21290.0,87.0,6.0,20.0,0.0,50.0,Fluxo,Blue,7.0
1,Viego,Sting,JUNGLE,13.0,1.0,2.0,3.0,2.0,179.0,165.0,...,184.0,29574.0,54.0,8.0,25.0,0.0,0.0,Fluxo,Blue,7.0
2,Taliyah,Fuuu,MID,15.0,1.0,3.0,3.0,1.3,264.0,8.0,...,551.0,16348.0,90.0,10.0,29.0,150.0,100.0,Fluxo,Blue,7.0
3,Varus,Kojima,ADC,13.0,1.0,4.0,3.0,1.0,230.0,0.0,...,311.0,15554.0,142.0,7.0,23.0,0.0,0.0,Fluxo,Blue,7.0
4,Nautilus,scamber,SUPPORT,10.0,0.0,2.0,4.0,2.0,36.0,0.0,...,185.0,12483.0,57.0,18.0,28.0,0.0,0.0,Fluxo,Blue,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,Gwen,Lonely,TOP,15.0,1.0,4.0,2.0,0.8,248.0,4.0,...,58.0,21346.0,80.0,10.0,23.0,0.0,0.0,KaBuM! e-Sports,Red,1.0
696,Amumu,Malrang,JUNGLE,12.0,0.0,4.0,3.0,0.8,142.0,129.0,...,489.0,20966.0,94.0,9.0,25.0,0.0,0.0,KaBuM! e-Sports,Red,1.0
697,Tristana,Hauz,MID,15.0,1.0,4.0,1.0,0.5,261.0,12.0,...,94.0,15257.0,100.0,5.0,19.0,0.0,50.0,KaBuM! e-Sports,Red,1.0
698,Draven,Netuno,ADC,13.0,1.0,2.0,2.0,1.5,259.0,0.0,...,71.0,11322.0,53.0,6.0,24.0,0.0,0.0,KaBuM! e-Sports,Red,1.0


In [4]:
all_games_data.groupby(['Player','Role'])['Solo kills'].sum().sort_values(ascending=False).head(3)

Player  Role
Makes   TOP     11.0
Guigo   TOP     10.0
fNb     TOP      7.0
Name: Solo kills, dtype: float64

In [5]:
all_games_data.groupby(['Player','Role'])['Solo kills'].mean().sort_values(ascending=False).head()

Player   Role
Makes    TOP     0.785714
Guigo    TOP     0.714286
fNb      TOP     0.500000
Fuuu     MID     0.428571
destroy  TOP     0.300000
Name: Solo kills, dtype: float64

In [6]:
all_games_data[['Player', 'Solo kills']].sort_values(by='Solo kills', ascending=False)

,Player,Solo kills
520,Makes,5.0
400,Guigo,3.0
530,SuperCleber,2.0
104,zay,2.0
505,Guigo,2.0
...,...,...
245,Makes,0.0
246,Drakehero,0.0
247,Pilot,0.0
248,micaO,0.0


In [9]:
no_senna = all_games_data.loc[all_games_data['Champions'] != 'Senna']

In [42]:
data= no_senna.loc[no_senna['Role'].isin(['ADC', 'MID', 'TOP'])].groupby(['Player', 'Role'])[['CSM', 'DPM']].mean().reset_index()

In [50]:
import plotly.express as px

fig = px.scatter(data_frame=data, x='CSM', y='DPM', color= 'Role',
                 symbol='Role', hover_name='Player', width=1000,)

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, plot_bgcolor='white')
fig.update_traces(marker_size=10)
fig.show()